# Optimization Methods - Step by Step

We'll implement and compare the permance of three well known optimization methods: Gradient Descent, Momentum and Adam.

**Notations**:  $\frac{\partial J}{\partial a } = $ `da` for any variable `a`.

<a name='0'></a>
## Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math
import sklearn
import sklearn.datasets

from dnn_utils import *
from opt_utils import predict_dec, plot_decision_boundary, load_dataset
from copy import deepcopy

%matplotlib inline
plt.rcParams['figure.figsize'] = (7,4) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

<a name='2'></a>
## 2 - Gradient Descent

the most common optimization method in machine learning is gradient descent (GD). When you take gradient steps with respect to all $m$ examples on each step, it is also called Batch Gradient Descent. 

The  gradient descent rule is, for $l = 1, ..., L$: 
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{1}$$


$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{2}$$

where L is the number of layers and $\alpha$ is the learning rate.

In [ ]:
def update_parameters_gd(parameters, grads, learning_rate):
    """
    Update parameters using one step of gradient descent
    
    Arguments:
    parameters -- python dictionary containing the parameters to be updated:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing the gradients to update each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    learning_rate -- scalar, the learning rate.
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    L = len(parameters) // 2

    # Updating rule for each parameter
    for l in range(1, L + 1):
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate*grads['dW' + str(l)]
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate*grads['db' + str(l)]

    return parameters

<a name='2.1'></a>
### 2.1 - Stochastic Gradient Descent

A variant of this method is the Stochastic Gradient Descent (SGD). In SGD, you use only 1 training example before updating the gradients. When the training set is large, SGD can be faster. But the parameters will "oscillate" toward the minimum rather than converge smoothly. Here's what that looks like:

<center> <img src="images/kiank_sgd.png" width="65%" height="65%"> </center>
<caption> <p> <b>Figure 1</b>: SGD vs GD. SGD leads to many oscillations to reach convergence, but each step is a lot faster to compute for SGD than it is for GD, as it uses only one training example. </p> </caption>
    
    
- **Gradient Descent**:
``` python
for i in range(0, num_iterations):
    # Forward propagation
    a, caches = forward_propagation(X, parameters)
    
    # Compute cost
    cost_total = compute_cost(a, Y)
    
    # Backward propagation
    grads = backward_propagation(a, caches, parameters)
    
    # Update parameters
    parameters = update_parameters(parameters, grads)
    
    # Compute average cost
    cost_avg = cost_total / a     
```


- **Stochastic Gradient Descent**:
```python
for i in range(0, num_iterations):
    cost_total = 0
    
    for j in range(0, m):
        # Forward propagation
        a, caches = forward_propagation(X[:,j], parameters)
        
        # Compute cost
        cost_total += compute_cost(a, Y[:,j])
        
        # Backward propagation
        grads = backward_propagation(a, caches, parameters)
        
        # Update parameters
        parameters = update_parameters(parameters, grads)
        
    # Compute average cost
    cost_avg = cost_total / m
```


<a name='2.2'></a>
### 2.2 - Mini-Batch Gradient Descent

In practice, you'll often get faster results if you don't use the entire training set, or just one training example, to perform each update. Mini-batch gradient descent uses an intermediate number of examples for each step. With mini-batch gradient descent, you loop over the mini-batches instead of looping over individual training examples.

<center> <img src="images/kiank_minibatch.png" width="65%" height="65%"> </center>
<caption> <center> <b>Figure 2</b>: SGD vs Mini-Batch GD. Using mini-batches in your optimization algorithm often leads to faster optimization. </center></caption>

In [ ]:
def random_mini_batches(X, Y, mini_batch_size=64, seed=0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- label vector of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            
    m = X.shape[1]             
    mini_batches = []
        
    # Shuffle the dataset 
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1, m))
    
    inc = mini_batch_size

    # Mini-batches partitioning
    num_complete_minibatches = math.floor(m/mini_batch_size)
    
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k*inc:(k+1)*inc]
        mini_batch_Y = shuffled_Y[:, k*inc:(k+1)*inc]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the case last mini-batch < mini_batch_size
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:,(k+1)*inc:m]
        mini_batch_Y = shuffled_Y[:,(k+1)*inc:m]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

<a name='3'></a>
## 3 - Momentum

As mini-batch gradient descent makes a parameter update after seeing just a subset of examples, the direction of the update has some variance, making the path "oscillate" toward convergence. Using momentum can reduce these oscillations. 

This optimization method takes into account the past gradients to smooth out the update. The 'direction' of the previous gradients is stored in the variable $v$. Formally, this will be the exponentially weighted average of the gradient on previous steps.

The momentum update rule is, for $l = 1, ..., L$: 

$$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
\\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}\tag{3}$$
<br>

$$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
\\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}\tag{4}$$

where L is the number of layers, $\beta$ is the momentum and $\alpha$ is the learning rate.

In [ ]:
def initialize_velocity(parameters):
    """
    Initializes the velocity as a python dictionary with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters
    Arguments:
    parameters -- python dictionary containing the parameters.
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    
    Returns:
    v -- python dictionary containing the current velocity.
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    """
    
    L = len(parameters) // 2
    v = {}
    
    # Initializing velocity
    for l in range(1, L + 1):
        v["dW" + str(l)] = np.zeros((parameters['W' + str(l)].shape[0], parameters['W' + str(l)].shape[1]))
        v["db" + str(l)] = np.zeros((parameters['b' + str(l)].shape[0], parameters['b' + str(l)].shape[1]))
        
    return v

In [ ]:
def update_parameters_momentum(parameters, grads, v, beta, learning_rate):
    """
    Update parameters using Momentum
    
    Arguments:
    parameters -- python dictionary containing the parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing the gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- python dictionary containing the current velocity:
                    v['dW' + str(l)] = ...
                    v['db' + str(l)] = ...
    beta -- the momentum hyperparameter, scalar
    learning_rate -- the learning rate, scalar
    
    Returns:
    parameters -- python dictionary containing the updated parameters 
    v -- python dictionary containing the updated velocities
    """

    L = len(parameters) // 2
    
    # Momentum update for each parameter
    for l in range(1, L + 1):
        v["dW" + str(l)] = beta*v["dW" + str(l)] + (1 - beta)*grads['dW' + str(l)]
        v["db" + str(l)] = beta*v["db" + str(l)] + (1 - beta)*grads['db' + str(l)]
        
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate*v["dW" + str(l)]
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate*v["db" + str(l)]
        
    return parameters, v

**Note that**:
- The velocity is initialized with zeros. So the algorithm will take a few iterations to "build up" velocity and start to take bigger steps.
- If $\beta = 0$, then this just becomes standard gradient descent without momentum. 

**How do you choose $\beta$?**

- The larger the momentum $\beta$ is, the smoother the update, because it takes the past gradients into account more. But if $\beta$ is too big, it could also smooth out the updates too much. 
- Common values for $\beta$ range from 0.8 to 0.999. If you don't feel inclined to tune this, $\beta = 0.9$ is often a reasonable default. 

<a name='4'></a>   
## 4 - Adam

Adam is one of the most effective optimization algorithms for training neural networks. It combines ideas from RMSProp  and Momentum. 

1. It calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v'$ (with bias correction);

2. It calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s'$ (with bias correction);

3. It updates parameters in a direction based on combining information from steps 1 and 2.

The update rule is, for $l = 1, ..., L$: 

$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \ dW^{[l]} \\
\\
v'_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t}
\end{cases}\tag{5}$$
<br>

$$\begin{cases}
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (dW^{[l]})^2 \\
\\
s'_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t}
\end{cases}\tag{6}$$
<br>

$$W^{[l]} = W^{[l]} - \alpha \frac{v'_{dW^{[l]}}}{\sqrt{s'_{dW^{[l]}}} + \varepsilon} \tag{7}$$

where:
- t counts the number of steps taken of Adam 
- L is the number of layers
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages
- $\alpha$ is the learning rate
- $\varepsilon$ is a very small number to avoid dividing by zero

In [ ]:
def initialize_adam(parameters) :
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    
    Arguments:
    parameters -- python dictionary containing the parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v -- python dictionary that will contain the exponentially weighted average of the gradient. Initialized with zeros.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient. Initialized with zeros.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...

    """
    
    L = len(parameters) // 2
    v = {}
    s = {}
    
    # Initializing v, s
    for l in range(1, L + 1):
        v["dW" + str(l)] = np.zeros((parameters['W' + str(l)].shape[0], parameters['W' + str(l)].shape[1]))
        v["db" + str(l)] = np.zeros((parameters['b' + str(l)].shape[0], parameters['b' + str(l)].shape[1]))
        
        s["dW" + str(l)] = np.zeros((parameters['W' + str(l)].shape[0], parameters['W' + str(l)].shape[1]))
        s["db" + str(l)] = np.zeros((parameters['b' + str(l)].shape[0], parameters['b' + str(l)].shape[1]))
    
    return v, s

In [ ]:
def update_parameters_adam(parameters, grads, v, s, t, learning_rate = 0.01, beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update parameters using Adam
    
    Arguments:
    parameters -- python dictionary containing the parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing the gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    t -- Adam variable, counts the number of taken steps
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing the updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    
    L = len(parameters) // 2                 
    v_corrected = {}                         
    s_corrected = {}                         
    
    # Adam update for each parameter
    for l in range(1, L + 1):
        v["dW" + str(l)] = beta1*v["dW" + str(l)] + (1 - beta1)*grads['dW' + str(l)]
        v["db" + str(l)] = beta1*v["db" + str(l)] + (1 - beta1)*grads['db' + str(l)]        

        v_corrected["dW" + str(l)] = v["dW" + str(l)]/(1 - beta1**t)
        v_corrected["db" + str(l)] = v["db" + str(l)]/(1 - beta1**t)

        s["dW" + str(l)] = beta2*s["dW" + str(l)] + (1 - beta2)*np.square(grads['dW' + str(l)])
        s["db" + str(l)] = beta2*s["db" + str(l)] + (1 - beta2)*np.square(grads['db' + str(l)])

        s_corrected["dW" + str(l)] = s["dW" + str(l)]/(1 - beta2**t)
        s_corrected["db" + str(l)] = s["db" + str(l)]/(1 - beta2**t)

        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate*v_corrected["dW" + str(l)]/(np.sqrt(s_corrected["dW" + str(l)]) + epsilon)
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate*v_corrected["db" + str(l)]/(np.sqrt(s_corrected["db" + str(l)]) + epsilon)

    return parameters, v, s, v_corrected, s_corrected

<a name='5'></a>  
## 5 - Testing the Optimization Algorithms

We'll use the "moons" dataset from **Scikit Learn** to test and compare the different optimization methods.

In [ ]:
train_X, train_Y = load_dataset()

We'll use a n-layer neural network implementation as our model and train it with the functions implemented above for each optimization method.

In [ ]:
def model(X, Y, layers_dims, initialization, optimizer, learning_rate = 0.001, mini_batch_size = 64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 5000, print_cost = True):
    """
    n-layer neural network model which can be run in different optimizer modes.
    
    Arguments:
    X -- input data, of shape (2, number of examples)
    Y -- label vector of shape (1, number of examples)
    initialization -- the initialization to be passed, random or he
    optimizer -- the optimizer to be passed, gradient descent, momentum or adam
    layers_dims -- python list, containing the size of each layer
    learning_rate -- the learning rate, scalar
    mini_batch_size -- the size of a mini batch
    beta -- Momentum hyperparameter
    beta1 -- Exponential decay hyperparameter for the past gradients estimates 
    beta2 -- Exponential decay hyperparameter for the past squared gradients estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates
    num_epochs -- number of epochs
    print_cost -- True to print the cost every 1000 epochs

    Returns:
    parameters -- python dictionary containing your updated parameters 
    """

    L = len(layers_dims)             
    costs = []                       
    epochs = []
    t = 0                                            
    m = X.shape[1]               
    seed = 0  
    
    # Initializing parameters
    parameters = initialize_parameters(layers_dims, initialization)

    # Initializing the optimizer
    if optimizer == "gd":
        pass # no initialization required for gradient descent
    
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
        
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    # Optimization loop
    for i in range(num_epochs):
        # Defining the random minibatches
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
        seed = seed + 1
        
        total_cost = 0
        for minibatch in minibatches:
            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch

            # Forward propagation
            AL, caches = forward_pass(minibatch_X, parameters)

            # Compute cost and add to the total cost
            total_cost += compute_cost(AL, minibatch_Y)

            # Backward propagation
            grads = backward_pass(AL, minibatch_Y, caches)

            # Update parameters
            if optimizer == "gd":
                parameters = update_parameters_gd(parameters, grads, learning_rate)
                
            elif optimizer == "momentum":
                parameters, v = update_parameters_momentum(parameters, grads, v, beta, learning_rate)
                
            elif optimizer == "adam":
                t = t + 1 # Adam counter
                parameters, v, s, _, _ = update_parameters_adam(parameters, grads, v, s, t, learning_rate, beta1, beta2, epsilon)
        
        cost_avg = total_cost/m
        
        # Saving the cost every 100 epoch
        if print_cost and i % 100 == 0:
            costs.append(cost_avg)
            
        # Printing the cost every 1000 epoch
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost_avg))
             
    # Ploting the cost
    plt.figure(figsize=(6,4))
    plt.plot(list(range(0, num_epochs, 100)), costs)
    plt.ylabel('Cost')
    plt.xlabel('Epochs')
    plt.title("Learning rate = " + str(learning_rate))
    
    ax = plt.gca()
    ax.set_xlim(0, num_epochs)
    
    plt.show()

    return parameters

In [ ]:
def predict(X, Y, parameters):
    """
    This function is used to predict the results of a n-layer neural network.
    
    Arguments:
    X -- dataset of examples
    Y -- labels vector
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    p = np.zeros((1,m), dtype = np.int32)
    
    # Forward propagation
    AL, caches = forward_pass(X, parameters)
    
    # convert probas to 0/1 predictions
    p[AL > 0.5] = 1

    # print results
    #print ("predictions: " + str(p[0,:]))
    #print ("true labels: " + str(Y[0,:]))
    acc = np.round(np.mean((p[0,:] == Y[0,:])), 3)
    print('Model accuracy: '  + str(acc))
    
    return p

<a name='5-1'></a>  
### 5.1 - Mini-Batch Gradient Descent

Run the following code to see how the model does with mini-batch gradient descent.

In [ ]:
# train n-layer model
layers_dims = [train_X.shape[0], 5, 5, 1]
parameters = model(train_X, train_Y, layers_dims, initialization='he', optimizer='gd')

# Plot decision boundary
plt.figure(figsize=(6,4))
plt.title("Model with Gradient Descent optimization")
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

# Predict
predictions = predict(train_X, train_Y, parameters)

<a name='5-2'></a>  
### 5.2 - Mini-Batch Gradient Descent with Momentum

Next, run the following code to see how the model does with momentum. Because this example is relatively simple, the gains from using momemtum are small - but for more complex problems you might see bigger gains.

In [ ]:
# train n-layer model
layers_dims = [train_X.shape[0], 5, 5, 1]
parameters = model(train_X, train_Y, layers_dims, initialization='he', beta = 0.9, optimizer = "momentum")

# Plot decision boundary
plt.figure(figsize=(6,4))
plt.title("Model with Momentum optimization")
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

# Predict
predictions = predict(train_X, train_Y, parameters)

<a name='5-3'></a>  
### 5.3 - Mini-Batch with Adam

Finally, run the following code to see how the model does with Adam.

In [ ]:
# train n-layer model
layers_dims = [train_X.shape[0], 5, 5, 1]
parameters = model(train_X, train_Y, layers_dims, initialization='he', optimizer="adam")

# Plot decision boundary
plt.figure(figsize=(6,4))
plt.title("Model with Adam optimization")
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

# Predict
predictions = predict(train_X, train_Y, parameters)

<a name='5-4'></a>  
### 5.4 - Summary

<table> 
    <tr>
        <td>
        <b>optimization method</b>
        </td>
        <td>
        <b>accuracy</b>
        </td>
        <td>
        <b>cost shape</b>
        </td>
    </tr>
        <td>
        Gradient descent
        </td>
        <td>
        >85%
        </td>
        <td>
        smooth
        </td>
    <tr>
        <td>
        Momentum
        </td>
        <td>
        >85%
        </td>
        <td>
        smooth
        </td>
    </tr>
    <tr>
        <td>
        Adam
        </td>
        <td>
         98%
        </td>
        <td>
        smoother
        </td>
    </tr>
</table> 


Momentum usually helps, but given the small learning rate and the simplistic dataset, its impact is almost negligible.

On the other hand, Adam clearly outperforms mini-batch gradient descent and Momentum. If you run the model for more epochs on this simple dataset, all three methods will lead to very good results. However, you've seen that Adam converges a lot faster.

<a name='6'></a>  
## 6 - Learning Rate Decay and Scheduling

Lastly, the learning rate is another hyperparameter that can help you speed up learning. 

During the first part of training, the model can get away with taking large steps, but over time, using a fixed value for the learning rate alpha can cause the model to get stuck in a wide oscillation that never quite converges. But if we're slowly reducing the learning rate alpha over time, the model could then take smaller, slower steps that bring it closer to the minimum. This is the idea behind learning rate decay. 

Learning rate decay can be achieved by using either adaptive methods or pre-defined learning rate schedules. 

We'll apply scheduled learning rate decay to the previous n-layer neural network in three different optimizer modes and see how each one differs, as well as the effect of scheduling at different epochs. 

In [ ]:
def model(X, Y, layers_dims, initialization, optimizer, learning_rate0 = 0.001, decay_rate = 1, decay = False, 
          mini_batch_size = 64, beta = 0.9, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8, num_epochs = 5000, print_cost = True):
    """
    n-layer neural network model which can be run in different optimizer modes.
    
    Arguments:
    X -- input data, of shape (2, number of examples)
    Y -- label vector of shape (1, number of examples)
    initialization -- the initialization to be passed, random or he
    optimizer -- the optimizer to be passed, gradient descent, momentum or adam
    layers_dims -- python list, containing the size of each layer
    learning_rate0 -- inicial learning rate, scalar
    decay_rate -- learning rate decay, scalar
    decay -- learning rate decay method, function
    mini_batch_size -- the size of a mini batch
    beta -- Momentum hyperparameter
    beta1 -- Exponential decay hyperparameter for the past gradients estimates 
    beta2 -- Exponential decay hyperparameter for the past squared gradients estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates
    num_epochs -- number of epochs
    print_cost -- True to print the cost every 1000 epochs

    Returns:
    parameters -- python dictionary containing your updated parameters 
    """

    L = len(layers_dims)             
    costs = []                       
    epochs = []
    t = 0                                            
    m = X.shape[1]               
    seed = 0  
    
    # Initializing parameters
    parameters = initialize_parameters(layers_dims, initialization)

    # Initializing the optimizer
    if optimizer == "gd":
        pass # no initialization required for gradient descent
    
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
        
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    # Optimization loop
    learning_rate = learning_rate0
    for i in range(num_epochs):
        # Defining the random minibatches
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
        seed = seed + 1
        
        total_cost = 0
        for minibatch in minibatches:
            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch

            # Forward propagation
            AL, caches = forward_pass(minibatch_X, parameters)

            # Compute cost and add to the total cost
            total_cost += compute_cost(AL, minibatch_Y)

            # Backward propagation
            grads = backward_pass(AL, minibatch_Y, caches)

            # Update parameters
            if optimizer == "gd":
                parameters = update_parameters_gd(parameters, grads, learning_rate)
                
            elif optimizer == "momentum":
                parameters, v = update_parameters_momentum(parameters, grads, v, beta, learning_rate)
                
            elif optimizer == "adam":
                t = t + 1 # Adam counter
                parameters, v, s, _, _ = update_parameters_adam(parameters, grads, v, s, t, learning_rate, beta1, beta2, epsilon)
        
        cost_avg = total_cost/m
        
        if decay:
            learning_rate = decay(learning_rate0, i, decay_rate)
        
        # Saving the cost every 100 epoch
        if print_cost and i % 100 == 0:
            costs.append(cost_avg)
            
        # Printing the cost every 1000 epoch
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost_avg),'   ','Learning rate after epoch %i: %f' %(i, learning_rate))
             
    # Ploting the cost
    print('\n')
    plt.figure(figsize=(6,4))
    plt.plot(list(range(0, num_epochs, 100)), costs)
    plt.ylabel('Cost')
    plt.xlabel('Epochs')
    plt.title("Learning rate = " + str(learning_rate0) + " and Decay rate = " + str(decay_rate))
    
    ax = plt.gca()
    ax.set_xlim(0, num_epochs)
    
    plt.show()

    return parameters

<a name='6-1'></a>  
### 6.1 - Decay on every iteration  

We'll test one of the pre-defined schedules for learning rate decay, called exponential learning rate decay. It takes this mathematical form:

$$\alpha = \frac{\alpha_{0}}{1 + decayRate \times epochNumber} \tag{8}$$


In [ ]:
def update_lr(learning_rate0, epoch_num, decay_rate):
    """
    Update the learning rate using exponential weight decay.
    
    Arguments:
    learning_rate0 -- Initial learning rate, scalar
    epoch_num -- Epoch number, integer
    decay_rate -- Decay rate, scalar

    Returns:
    learning_rate -- Updated learning rate. Scalar 
    """

    learning_rate = learning_rate0/(1 + decay_rate*epoch_num)
    
    return learning_rate

In [ ]:
# train n-layer model
layers_dims = [train_X.shape[0], 5, 5, 1]

parameters = model(train_X, train_Y, layers_dims, initialization='he', optimizer='gd', learning_rate0=0.1, decay=update_lr, num_epochs=5000)

# Plot decision boundary
plt.figure(figsize=(6,4))
plt.title("Model with Gradient Descent optimization")
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

# Predict
predictions = predict(train_X, train_Y, parameters)

Notice that if you set the decay to occur at every iteration, the learning rate goes to zero too quickly - even if you start with a higher learning rate. 
<table> 
    <tr>
        <td>
        <b>Epoch Number</b>
        </td>
        <td>
        <b>Learning Rate</b>
        </td>
        <td>
        <b>Cost</b>
        </td>
    </tr>
    <tr>
        <td>
        0
        </td>
        <td>
        0.100000
        </td>
        <td>
        0.701091
        </td>
    </tr>
    <tr>
        <td>
        1000
        </td>
        <td>
        0.000100
        </td>
        <td>
        0.661884
        </td>
    </tr>
    <tr>
        <td>
        2000
        </td>
        <td>
        0.000050
        </td>
        <td>
        0.658620
        </td>
    </tr>
    <tr>
        <td>
        3000
        </td>
        <td>
        0.000033
        </td>
        <td>
        0.656765
        </td>
    </tr>
    <tr>
        <td>
        4000
        </td>
        <td>
        0.000025
        </td>
        <td>
        0.655486
        </td>
    </tr>
    <tr>
        <td>
        5000
        </td>
        <td>
        0.000020
        </td>
        <td>
        0.654514
        </td>
    </tr>
</table> 

When you're training for a few epochs this doesn't cause a lot of troubles, but when the number of epochs is large, the optimization algorithm will stop updating. One common fix to this issue is to decay the learning rate every few steps. This is called fixed interval scheduling.

<a name='-2'></a> 
### 6.2 - Fixed Interval Scheduling

You can either number the intervals or divide the epoch by the time interval, which is the size of the window with constant learning rate. 

$$\alpha = \frac{\alpha_{0}}{1 + decayRate \times \lfloor\frac{epochNum}{timeInterval}\rfloor} \tag{9}$$

In [ ]:
def schedule_lr_decay(learning_rate0, epoch_num, decay_rate, time_interval=1000):
    """
    Update the learning rate using exponential weight decay.
    
    Arguments:
    learning_rate0 -- Initial learning rate, scalar
    epoch_num -- Epoch number, integer
    decay_rate -- Decay rate, scalar
    time_interval -- Number of epochs to update the learning rate, scalar

    Returns:
    learning_rate -- Updated learning rate, scalar 
    """
 
    learning_rate = learning_rate0/(1 + decay_rate*math.floor(epoch_num/time_interval))
    
    return learning_rate

<a name='6-2-1'></a> 
#### 6.2.1 - Gradient Descent with Learning Rate Decay

In [ ]:
# train n-layer model
layers_dims = [train_X.shape[0], 5, 5, 1]
parameters = model(train_X, train_Y, layers_dims, initialization='he', optimizer="gd", learning_rate0=0.1, num_epochs=5000, decay=schedule_lr_decay)

# Plot decision boundary
plt.figure(figsize=(6,4))
plt.title("Model with Gradient Descent optimization")
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

# Predict
predictions = predict(train_X, train_Y, parameters)

<a name='6-2-2'></a> 
#### 6.3.2 - Gradient Descent with Momentum and Learning Rate Decay

In [ ]:
# train n-layer model
layers_dims = [train_X.shape[0], 5, 5, 1]
parameters = model(train_X, train_Y, layers_dims, initialization='he', optimizer="momentum", learning_rate0=0.1, num_epochs=5000, decay=schedule_lr_decay)

# Plot decision boundary
plt.figure(figsize=(6,4))
plt.title("Model with Momentum optimization")
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

# Predict
predictions = predict(train_X, train_Y, parameters)

<a name='6-4'></a> 
### 6.4 - Achieving similar performance with different methods

With Mini-batch GD or Mini-batch GD with Momentum, the accuracy is significantly lower than Adam, but when learning rate decay is added on top, either can achieve performance at a speed and accuracy score that's similar to Adam.

In the case of Adam, notice that the learning curve achieves a similar accuracy but faster.

<table> 
    <tr>
        <td>
        <b>optimization method</b>
        </td>
        <td>
        <b>accuracy</b>
        </td>
    </tr>
        <td>
        Gradient descent
        </td>
        <td>
        98%
        </td>
    <tr>
        <td>
        Momentum
        </td>
        <td>
        99%
        </td>
    </tr>
    <tr>
        <td>
        Adam
        </td>
        <td>
        98%
        </td>
    </tr>
</table> 